# Solving clues with DNN!
In this notebook, I outline the process through which the model is able to solve clues.

In [4]:
import drqa
from nltk.parse.corenlp import CoreNLPParser
from drqa.tokenizers import SimpleTokenizer, CoreNLPTokenizer
drqa.tokenizers.set_default('corenlp_classpath', '../lib/DrQA/data/corenlp/*')

ModuleNotFoundError: No module named 'elasticsearch'

In [2]:
from drqa.tokenizers import CoreNLPTokenizer
tok = CoreNLPTokenizer(mem='2g')
tok.tokenize("Hello world").words()  # Should complete immediately

['Hello', 'world']

In [4]:
from sklearn.model_selection import train_test_split
import pandas as pd

df = pd.read_csv('../data/puns.csv',header=None,names=['answer','clue'])
df

,answer,clue
0,AAA,Big gp. of towers?
1,AAA,"Great ""Three R's"" report card?"
2,AAA,Org. that can get you discounts on mortgages a...
3,AAAA,"Fonzi's report card, so he says?"
4,AAH,Sound heard from a Jacuzzi?
...,...,...
15211,ZOO,"Scorpions song about animal exhibition (with ""..."
15212,ZOOKEEPER,"Big cat person, maybe?"
15213,ZORBATHEGRIEG,"Quinn, Norwegian style?"
15214,ZOXHZUHEHE,"Do the ""I am not a crook"" thing with the doubl..."


In [6]:
from drqa import pipeline

# from drqa.scripts

In [7]:
word_list = pd.read_csv('../data/word_list.csv',sep=';',header=None,names=['word','score'])

In [8]:
word_list

,word,score
0,03BONNIEANDCLYDE,56
1,0AD,25
2,0BC,25
3,0TH,75
4,100,5
...,...,...
567650,ZZYZWICZ,50
567651,ZZYZX,48
567652,ZZZ,30
567653,ZZZQUIL,50


In [9]:
import regex

In [10]:
no_nums_wl = word_list[~word_list['word'].str.contains(r'\d').astype(bool)]
correct_length_wl = no_nums_wl[no_nums_wl['word'].str.len() >=3]
correct_length_wl = correct_length_wl[correct_length_wl['word'].str.len()<=15]

In [11]:
correct_length_wl

,word,score
489,AAA,50
490,AAAA,30
491,AAAAH,25
492,AAAAUTOCLUB,50
493,AAABALL,75
...,...,...
567650,ZZYZWICZ,50
567651,ZZYZX,48
567652,ZZZ,30
567653,ZZZQUIL,50


In [12]:
%load ../src/dp_wordbreak.py


In [13]:
%load?

import ../src/dp_wordbreak

SyntaxError: invalid syntax (<ipython-input-13-2dab1a161a4c>, line 3)

In [1]:
import sys
sys.path.insert(0,'/home/thowe/proj_ssd/shor.tz/src/')

In [2]:
from dp_wordbreak import Trie, TrieNode, WordBreaker

In [3]:
wb = WordBreaker()

In [4]:
wb.word_break("hell",['he','ll'])


True

In [6]:
from typing import List


In [8]:
wordBreak("hell",['he','ll'])

TypeError: wordBreak() missing 1 required positional argument: 'wordDict'

In [1]:
from datasets import load_dataset
ds = load_dataset("wikipedia","20220301.en")

Downloading:   0%|          | 0.00/15.3k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/20.3G [00:00<?, ?B/s]

Dataset wikipedia downloaded and prepared to /home/thowe/.cache/huggingface/datasets/wikipedia/20220301.en/2.0.0/aa542ed919df55cc5d3347f42dd4521d05ca68751f50dbc32bae2a7f1e167559. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
ds.save_to_disk('../data/wikipedia/ds_dict')

In [1]:
ds

NameError: name 'ds' is not defined

In [14]:
ds['train'].to_csv('../data/wikipedia/ds/wikipedia_ds.csv',num_proc=8)

Creating CSV from Arrow format:   0%|          | 0/646 [00:00<?, ?ba/s]

20278616363

In [1]:
from hashformers import TransformerWordSegmenter as WordSegmenter
import mxnet

ws = WordSegmenter(
    segmenter_model_name_or_path="gpt2",
    reranker_model_name_or_path="bert-base-uncased"
)

/home/thowe/miniconda3/envs/pytorch-rapids/lib/python3.9/site-packages/mxnet/optimizer/optimizer.py:163: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  warnings.warn('WARNING: New optimizer %s.%s is overriding '
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLMOptimized: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLMOptimized from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLMOptimized from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
import pandas as pd
clue_list = pd.read_csv('../data/cwdb/cwdb.txt',sep='\t', header=None, index_col=0, names=['id','clue','answer', 'date','publisher'])

In [12]:
clue_list_nyt = clue_list[clue_list['publisher'] == 'NYT']

In [2]:
import pickle

In [29]:
clue_list_nyt['answer'] = list(clue_list_nyt['answer'].apply(lambda x: str(x)))

/tmp/ipykernel_15464/2900793366.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clue_list_nyt['answer'] = list(clue_list_nyt['answer'].apply(lambda x: str(x)))


In [30]:
clue_list_nyt['list_segmented'] = ws.segment(word_list=clue_list_nyt['answer'][:len(clue_list_nyt)])

/tmp/ipykernel_15464/2338771256.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clue_list_nyt['list_segmented'] = ws.segment(word_list=clue_list_nyt['answer'][:len(clue_list_nyt)])


In [35]:
pickle.dump(obj=clue_list_nyt,file=open('../bin/split_words_nyt.pickle','wb'))

In [21]:

clue_list_nyt['answer'].dtype


dtype('O')

In [3]:
nyt_clue_df = pickle.load(file=open('../bin/split_words_nyt.pickle','rb'))

In [4]:
nyt_clue_df = nyt_clue_df.drop(['id',],axis=1)

In [5]:
nyt_clue_df

,clue,answer,date,publisher,list_segmented
1,Auto org.,aaa,1990-01-01,NYT,aaa
2,Auto club letters,aaa,1995-01-01,NYT,aaa
4,Org. that gets members reduced motel rates,aaa,1998-01-01,NYT,aaa
7,Car owners' org.,aaa,2000-01-01,NYT,aaa
16,Motorists' grp.,aaa,2004-01-01,NYT,aaa
...,...,...,...,...,...
2000489,Stomach,brook,1997-03-01,NYT,brook
2000490,"Symposium, originally",wineparty,1997-03-01,NYT,wineparty
2000491,Time-saving means of travel,sst,1997-03-01,NYT,sst
2000492,Went over the same ground?,rehoed,1997-03-01,NYT,rehoed


In [3]:
import pickle
import pandas as pd

In [4]:
nyt_clue_df = pickle.load(file=open('../bin/split_words_nyt.pickle','rb'))
type(nyt_clue_df)

pandas.core.frame.DataFrame

In [7]:
nyt_clue_df.to_csv('../data/clean/nyt_split_clues.csv',index=None)

In [2]:
nyt_clue_df = pd.read_csv('../data/clean/nyt_split_clues.csv')

NameError: name 'pd' is not defined

In [15]:
import drqa.reader
import drqa.tokenizers
reader = drqa.reader.Predictor()

In [23]:
from drqa.pipeline import DrQA

In [6]:
from transformers import BertForQuestionAnswering

model = BertForQuestionAnswering.from_pretrained('bert-large-uncased')

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForQuestionAnswering: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-large-uncased

In [7]:
from sklearn.model_selection import train_test_split
nyt_clue_df

,id,clue,answer,date,publisher,list_segmented
1,NYT_Jan01-1990,Auto org.,aaa,1990-01-01,NYT,aaa
2,NYT_Jan01-1995,Auto club letters,aaa,1995-01-01,NYT,aaa
4,NYT_Jan01-1998,Org. that gets members reduced motel rates,aaa,1998-01-01,NYT,aaa
7,NYT_Jan01-2000,Car owners' org.,aaa,2000-01-01,NYT,aaa
16,NYT_Jan01-2004,Motorists' grp.,aaa,2004-01-01,NYT,aaa
...,...,...,...,...,...,...
2000489,NYT_Mar01-1997,Stomach,brook,1997-03-01,NYT,brook
2000490,NYT_Mar01-1997,"Symposium, originally",wineparty,1997-03-01,NYT,wineparty
2000491,NYT_Mar01-1997,Time-saving means of travel,sst,1997-03-01,NYT,sst
2000492,NYT_Mar01-1997,Went over the same ground?,rehoed,1997-03-01,NYT,rehoed


In [13]:
!head ../data/word_list.csv

03BONNIEANDCLYDE;56
0AD;25
0BC;25
0TH;75
100;5
10000MANIACS;75
100GRANDBAR;50
100PERCENT;50
100POUNDSSOAKINGWET;75
100PROOFAGEDINSOUL;53


In [1]:
import pandas as pd
scored_words = pd.read_csv('../data/word_list.csv',sep=';',header=None,names=["word","score"])

In [7]:
scored_words['word'] = list(scored_words['word'].apply(str))

In [8]:
scored_words['word']=scored_words["word"].apply(str.lower)

In [ ]:
scored_words['words_seg'] = ws.segment(word_list=scored_words['word'][:len(scored_words)])

In [ ]:
scored_words.to_csv('../data/clean/word_list_seg.csv')

In [14]:
list = pd.read_csv('../data/clean/nyt_split_clues.csv')

In [15]:
list

,id,clue,answer,date,publisher,list_segmented
0,NYT_Jan01-1990,Auto org.,aaa,1990-01-01,NYT,aaa
1,NYT_Jan01-1995,Auto club letters,aaa,1995-01-01,NYT,aaa
2,NYT_Jan01-1998,Org. that gets members reduced motel rates,aaa,1998-01-01,NYT,aaa
3,NYT_Jan01-2000,Car owners' org.,aaa,2000-01-01,NYT,aaa
4,NYT_Jan01-2004,Motorists' grp.,aaa,2004-01-01,NYT,aaa
...,...,...,...,...,...,...
390381,NYT_Mar01-1997,Stomach,brook,1997-03-01,NYT,brook
390382,NYT_Mar01-1997,"Symposium, originally",wineparty,1997-03-01,NYT,wineparty
390383,NYT_Mar01-1997,Time-saving means of travel,sst,1997-03-01,NYT,sst
390384,NYT_Mar01-1997,Went over the same ground?,rehoed,1997-03-01,NYT,rehoed


In [16]:
list = list[['clue','list_segmented']]
list = pd.DataFrame(list)

In [22]:

list.columns = ['question','answer']
list['answer'] = list['answer'].apply(lambda x : [x])

In [24]:
the_list = list

In [27]:
the_list.to_json('../data/clean/json/ready_clues.json','records',lines=True)
!head ../data/clean/json/ready_clues.json

{"question":"Auto org.","answer":["aaa"]}
{"question":"Auto club letters","answer":["aaa"]}
{"question":"Org. that gets members reduced motel rates","answer":["aaa"]}
{"question":"Car owners' org.","answer":["aaa"]}
{"question":"Motorists' grp.","answer":["aaa"]}
{"question":"Tiny battery type","answer":["aaa"]}
{"question":"Duracell size","answer":["aaa"]}
{"question":"Org. that helps with tow service","answer":["aaa"]}
{"question":"Battery for a camera or phone","answer":["aaa"]}
{"question":"Kind of baseball or battery","answer":["aaa"]}


In [40]:
the_list[the_list['question'] == 'Above, with "of"']

,question,answer
342051,"Above, with ""of""",[upwards]


In [33]:
the_list

,question,answer
0,Auto org.,[aaa]
1,Auto club letters,[aaa]
2,Org. that gets members reduced motel rates,[aaa]
3,Car owners' org.,[aaa]
4,Motorists' grp.,[aaa]
...,...,...
390381,Stomach,[brook]
390382,"Symposium, originally",[wineparty]
390383,Time-saving means of travel,[sst]
390384,Went over the same ground?,[rehoed]
